In [ ]:
//mounting the google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


installing all needed libraries

In [2]:
!pip install pinecone-client torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [10]:
!pip install "pinecone-client[grpc]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone

# If interacting with Pinecone via HTTP requests, use:
# from pinecone import Pinecone

pc = Pinecone(api_key="011acd2f-5066-4061-bb01-d22b490ef756")


In [17]:
from pinecone import ServerlessSpec

index_name = "actors"
dimension = 2048  # Output dimension for ResNet-50
pc.create_index(index_name, dimension=dimension,metric="cosine",
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    ) )

In [18]:
import torch
from torchvision import models, transforms
from PIL import Image
import os

In [19]:
model = models.resnet50(pretrained=True)
model.eval()



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 67.8MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [26]:
def preprocess_image(file_path):
    img = Image.open(file_path).convert('RGB')
    img_data = preprocess(img)
    img_data = img_data.unsqueeze(0)  # Add batch dimension
    return img_data

# Function to extract features using ResNet
def extract_features(model, img_data):
    with torch.no_grad():
        features = model.avgpool(model.forward_features(img_data))
        features = features.squeeze()  # Remove unnecessary dimensions
    return features

In [29]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install Necessary Libraries
!pip install pinecone-client torch torchvision

# Step 3: Initialize Pinecone
import pinecone
from pinecone.grpc import PineconeGRPC as Pinecone


PINECONE_API_KEY = "011acd2f-5066-4061-bb01-d22b490ef756"
pc = Pinecone(api_key=PINECONE_API_KEY)

# Step 4: Create Pinecone Index
index_name = "actors"
# dimension = 2048  # Output dimension for ResNet-50
# pinecone.create_index(index_name, dimension=dimension)
index = pc.Index(index_name)

# Step 5: Load and Preprocess Images, Extract Features with ResNet, and Upload to Pinecone
import torch
from torchvision import models, transforms
from PIL import Image
import os

# Initialize ResNet model
model = models.resnet50(pretrained=True)
model.eval()

# Modify ResNet to output features before the classification layer
class ResNetFeatureExtractor(torch.nn.Module):
    def __init__(self, model):
        super(ResNetFeatureExtractor, self).__init__()
        self.features = torch.nn.Sequential(*list(model.children())[:-2])  # Remove the last FC layer and avgpool
        self.avgpool = model.avgpool

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)  # Flatten the output
        return x

# Wrap the model
feature_extractor = ResNetFeatureExtractor(model)

# Preprocess image function
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to preprocess image
def preprocess_image(file_path):
    img = Image.open(file_path).convert('RGB')
    img_data = preprocess(img)
    img_data = img_data.unsqueeze(0)  # Add batch dimension
    return img_data

# Function to extract features using the modified ResNet model
def extract_features(model, img_data):
    with torch.no_grad():
        features = model(img_data)
    return features

# Directory containing images in Google Drive
image_dir = '/content/drive/My Drive/actors/mamooty'  # Replace with your folder path

# Process files and extract features
for file_name in os.listdir(image_dir):
    file_path = os.path.join(image_dir, file_name)

    # Check if file is an image
    if file_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
        # Preprocess image
        img_data = preprocess_image(file_path)

        # Extract features
        features = extract_features(feature_extractor, img_data)

        # Convert features to a list
        vector = features.flatten().tolist()

        # Add vector to Pinecone
        index.upsert([{"id": "mamooty", "values": vector}])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
image_dir = '/content/drive/My Drive/actors/mohanlal'  # Replace with your folder path

# Process files and extract features
for file_name in os.listdir(image_dir):
    file_path = os.path.join(image_dir, file_name)

    # Check if file is an image
    if file_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
        # Preprocess image
        img_data = preprocess_image(file_path)

        # Extract features
        features = extract_features(feature_extractor, img_data)

        # Convert features to a list
        vector = features.flatten().tolist()

        # Add vector to Pinecone
        index.upsert([{"id": "mohanlal", "values": vector}])


In [44]:
image_dir = '/content/drive/MyDrive/actors/testcase/download (6).jpeg'
img_data = preprocess_image(file_path)

features = extract_features(feature_extractor, img_data)
vector = features.flatten().tolist()
print(vector)



[0.08769864588975906, 1.7271438837051392, 0.6573916077613831, 0.2792373597621918, 0.22327594459056854, 0.24217739701271057, 2.401843547821045, 0.27447977662086487, 0.04151449725031853, 0.28206896781921387, 0.09330777078866959, 0.3470344543457031, 0.31818464398384094, 0.3375565707683563, 0.640835165977478, 0.25703561305999756, 0.5432628989219666, 0.3719707727432251, 0.023451415821909904, 0.4948103725910187, 0.011556800454854965, 0.09677085280418396, 0.1374598741531372, 0.04184912517666817, 0.016603993251919746, 0.12376594543457031, 0.4836547374725342, 0.1812250167131424, 0.5144144892692566, 0.03634442016482353, 0.06080077961087227, 0.4780025780200958, 0.5514498949050903, 1.4281795024871826, 0.5297769904136658, 0.053921714425086975, 0.25933974981307983, 0.13625676929950714, 0.10084202885627747, 0.03865734115242958, 0.2813173830509186, 0.6965866088867188, 0.04732980579137802, 0.3776022791862488, 0.2060115784406662, 0.3211517632007599, 0.004088236950337887, 1.0929131507873535, 0.6146215200

In [43]:
index.query(

    vector=vector,
    top_k=10,
   # include_values=True
)

{'matches': [{'id': 'mohanlal',
              'metadata': None,
              'score': 1.0003673,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'mamooty',
              'metadata': None,
              'score': 0.7994939,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}